In [3]:
from ibapi.wrapper import EWrapper
from ibapi.client import EClient
from ibapi.contract import Contract as IBcontract
from threading import Thread
import pandas as pd
import queue
import datetime

In [4]:
DEFAULT_HISTORIC_DATA_ID=50
DEFAULT_GET_CONTRACT_ID=43

## marker for when queue is finished
FINISHED = object()
STARTED = object()
TIME_OUT = object()

class finishableQueue(object):

    def __init__(self, queue_to_finish):

        self._queue = queue_to_finish
        self.status = STARTED

    def get(self, timeout):
        """
        Returns a list of queue elements once timeout is finished, or a FINISHED flag is received in the queue
        :param timeout: how long to wait before giving up
        :return: list of queue elements
        """
        contents_of_queue=[]
        finished=False

        while not finished:
            try:
                current_element = self._queue.get(timeout=timeout)
                if current_element is FINISHED:
                    finished = True
                    self.status = FINISHED
                else:
                    contents_of_queue.append(current_element)
                    ## keep going and try and get more data

            except queue.Empty:
                ## If we hit a time out it's most probable we're not getting a finished element any time soon
                ## give up and return what we have
                finished = True
                self.status = TIME_OUT


        return contents_of_queue

    def timed_out(self):
        return self.status is TIME_OUT





class TestWrapper(EWrapper):
    """
    The wrapper deals with the action coming back from the IB gateway or TWS instance
    We override methods in EWrapper that will get called when this action happens, like currentTime
    Extra methods are added as we need to store the results in this object
    """

    def __init__(self):
        self._my_contract_details = {}
        self._my_historic_data_dict = {}

    ## error handling code
    def init_error(self):
        error_queue=queue.Queue()
        self._my_errors = error_queue

    def get_error(self, timeout=5):
        if self.is_error():
            try:
                return self._my_errors.get(timeout=timeout)
            except queue.Empty:
                return None

        return None

    def is_error(self):
        an_error_if=not self._my_errors.empty()
        return an_error_if

    def error(self, id, errorCode, errorString):
        ## Overriden method
        errormsg = "IB error id %d errorcode %d string %s" % (id, errorCode, errorString)
        self._my_errors.put(errormsg)


    ## get contract details code
    def init_contractdetails(self, reqId):
        contract_details_queue = self._my_contract_details[reqId] = queue.Queue()

        return contract_details_queue

    def contractDetails(self, reqId, contractDetails):
        ## overridden method

        if reqId not in self._my_contract_details.keys():
            self.init_contractdetails(reqId)

        self._my_contract_details[reqId].put(contractDetails)

    def contractDetailsEnd(self, reqId):
        ## overriden method
        if reqId not in self._my_contract_details.keys():
            self.init_contractdetails(reqId)

        self._my_contract_details[reqId].put(FINISHED)

    ## Historic data code
    def init_historicprices(self, tickerid):
        historic_data_queue = self._my_historic_data_dict[tickerid] = queue.Queue()

        return historic_data_queue


    def historicalData(self, tickerid , bar):

        ## Overriden method
        ## Note I'm choosing to ignore barCount, WAP and hasGaps but you could use them if you like
        bardata=(bar.date, bar.open, bar.high, bar.low, bar.close, bar.volume)

        historic_data_dict=self._my_historic_data_dict

        ## Add on to the current data
        if tickerid not in historic_data_dict.keys():
            self.init_historicprices(tickerid)

        historic_data_dict[tickerid].put(bardata)

    def historicalDataEnd(self, tickerid, start:str, end:str):
        ## overriden method

        if tickerid not in self._my_historic_data_dict.keys():
            self.init_historicprices(tickerid)

        self._my_historic_data_dict[tickerid].put(FINISHED)




class TestClient(EClient):
    """
    The client method
    We don't override native methods, but instead call them from our own wrappers
    """
    def __init__(self, wrapper):
        ## Set up with a wrapper inside
        EClient.__init__(self, wrapper)


    def resolve_ib_contract(self, ibcontract, reqId=DEFAULT_GET_CONTRACT_ID):

        """
        From a partially formed contract, returns a fully fledged version
        :returns fully resolved IB contract
        """

        ## Make a place to store the data we're going to return
        contract_details_queue = finishableQueue(self.init_contractdetails(reqId))

        print("Getting full contract details from the server... ")

        self.reqContractDetails(reqId, ibcontract)

        ## Run until we get a valid contract(s) or get bored waiting
        MAX_WAIT_SECONDS = 10
        new_contract_details = contract_details_queue.get(timeout = MAX_WAIT_SECONDS)

        while self.wrapper.is_error():
            print(self.get_error())

        if contract_details_queue.timed_out():
            print("Exceeded maximum wait for wrapper to confirm finished - seems to be normal behaviour")

        if len(new_contract_details)==0:
            print("Failed to get additional contract details: returning unresolved contract")
            return ibcontract

        if len(new_contract_details)>1:
            print("got multiple contracts using first one")

        new_contract_details=new_contract_details[0]

        resolved_ibcontract=new_contract_details.contract

        return resolved_ibcontract


    def get_IB_historical_data(self, ibcontract, durationStr="1 Y", barSizeSetting="5 mins",whatToShow='TRADES',useRTH=1,
                               tickerid=DEFAULT_HISTORIC_DATA_ID):

        """
        Returns historical prices for a contract, up to today
        ibcontract is a Contract
        :returns list of prices in 4 tuples: Open high low close volume
        """


        ## Make a place to store the data we're going to return
        historic_data_queue = finishableQueue(self.init_historicprices(tickerid))

        # Request some historical data. Native method in EClient
        self.reqHistoricalData(
            tickerid,  # tickerId,
            ibcontract,  # contract,
            datetime.datetime.today().strftime("%Y%m%d %H:%M:%S %Z"),  # endDateTime,
            durationStr,  # durationStr,
            barSizeSetting,  # barSizeSetting,
            "TRADES",  # whatToShow,
            1,  # useRTH,
            1,  # formatDate
            False,  # KeepUpToDate <<==== added for api 9.73.2
            [] ## chartoptions not used
        )



        ## Wait until we get a completed data, an error, or get bored waiting
        MAX_WAIT_SECONDS = 10
        print("Getting historical data from the server... could take %d seconds to complete " % MAX_WAIT_SECONDS)

        historic_data = historic_data_queue.get(timeout = MAX_WAIT_SECONDS)

        while self.wrapper.is_error():
            print(self.get_error())

        if historic_data_queue.timed_out():
            print("Exceeded maximum wait for wrapper to confirm finished - seems to be normal behaviour")

        self.cancelHistoricalData(tickerid)


        return historic_data



class TestApp(TestWrapper, TestClient):
    def __init__(self, ipaddress, portid, clientid):
        TestWrapper.__init__(self)
        TestClient.__init__(self, wrapper=self)

        self.connect(ipaddress, portid, clientid)

        thread = Thread(target = self.run)
        thread.start()

        setattr(self, "_thread", thread)

        self.init_error()


#if __name__ == '__main__':

app = TestApp("127.0.0.1", 4001, 1)

stocks = ['SPX']
historik = pd.DataFrame()
i=49
for stock in stocks:
    i+=1
    ibcontract = IBcontract()
    ibcontract.secType = "IND"
#ibcontract.lastTradeDateOrContractMonth="201809"
    ibcontract.symbol=stock
    ibcontract.currency="USD"
    ibcontract.exchange="CBOE"
    resolved_ibcontract=app.resolve_ib_contract(ibcontract)
    historic_data = app.get_IB_historical_data(resolved_ibcontract,tickerid=i)
  
    app.disconnect()
  

Getting full contract details from the server... 
Getting historical data from the server... could take 10 seconds to complete 
IB error id -1 errorcode 2106 string La connection de données HMDS est OK:ushmds.nj


In [5]:
historic_data

[('20180702  15:30:00', 2704.95, 2704.95, 2698.95, 2701.58, 0),
 ('20180702  15:35:00', 2701.65, 2706.0, 2701.64, 2705.99, 0),
 ('20180702  15:40:00', 2706.08, 2706.08, 2701.23, 2701.33, 0),
 ('20180702  15:45:00', 2701.39, 2705.29, 2700.77, 2704.19, 0),
 ('20180702  15:50:00', 2704.2, 2705.0, 2701.86, 2704.78, 0),
 ('20180702  15:55:00', 2704.73, 2707.22, 2702.51, 2707.18, 0),
 ('20180702  16:00:00', 2707.11, 2710.6, 2706.04, 2710.31, 0),
 ('20180702  16:05:00', 2710.55, 2710.77, 2707.83, 2709.14, 0),
 ('20180702  16:10:00', 2709.22, 2709.26, 2703.03, 2703.18, 0),
 ('20180702  16:15:00', 2703.26, 2706.18, 2703.26, 2705.7, 0),
 ('20180702  16:20:00', 2705.75, 2706.79, 2704.61, 2706.74, 0),
 ('20180702  16:25:00', 2706.81, 2709.52, 2706.81, 2709.38, 0),
 ('20180702  16:30:00', 2709.39, 2711.35, 2709.19, 2711.33, 0),
 ('20180702  16:35:00', 2711.38, 2712.15, 2710.08, 2712.15, 0),
 ('20180702  16:40:00', 2712.16, 2714.11, 2710.79, 2713.21, 0),
 ('20180702  16:45:00', 2713.22, 2713.65, 271

In [30]:
date_list=[]
open_list=[]
high_list=[]
low_list=[]
close_list=[]
volume_list=[]

for i in range (0,len(historic_data)):
    date_list.append(historic_data[i][0])
    open_list.append(historic_data[i][1])
    high_list.append(historic_data[i][2])
    low_list.append(historic_data[i][3])
    close_list.append(historic_data[i][4])
    volume_list.append(historic_data[i][5])

In [31]:
historik['Date']=date_list
historik['Open']=open_list
historik['High']=high_list
historik['Low']=low_list
historik['Close']=close_list
historik['Volume']=volume_list
historik.head()

,Date,Open,High,Low,Close,Volume
0,20180612 15:30:00,2785.60,2787.56,2785.42,2786.97,0
1,20180612 15:35:00,2786.94,2787.33,2784.51,2785.34,0
2,20180612 15:40:00,2785.39,2785.93,2784.15,2784.34,0
3,20180612 15:45:00,2784.33,2784.97,2783.12,2783.64,0
4,20180612 15:50:00,2783.61,2783.92,2782.10,2783.12,0


In [32]:

historik

,Date,Open,High,Low,Close,Volume
0,20180612 15:30:00,2785.60,2787.56,2785.42,2786.97,0
1,20180612 15:35:00,2786.94,2787.33,2784.51,2785.34,0
2,20180612 15:40:00,2785.39,2785.93,2784.15,2784.34,0
3,20180612 15:45:00,2784.33,2784.97,2783.12,2783.64,0
4,20180612 15:50:00,2783.61,2783.92,2782.10,2783.12,0
5,20180612 15:55:00,2783.09,2783.83,2782.46,2783.20,0
6,20180612 16:00:00,2783.36,2785.25,2782.93,2785.25,0
7,20180612 16:05:00,2785.23,2785.80,2785.14,2785.37,0
8,20180612 16:10:00,2785.36,2786.31,2785.35,2786.18,0
9,20180612 16:15:00,2786.19,2786.61,2784.54,2785.13,0


In [33]:
type(historik)

pandas.core.frame.DataFrame

In [34]:
historic_data[87][5]

0

In [35]:
len(historic_data)

19401

In [36]:
historik.to_csv('SPX_1Y_5mn.csv')
